# Рекомендация тарифов

<h1>Содержание:<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-и-изучите-файл" data-toc-modified-id="Откройте-и-изучите-файл-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте и изучите файл</a></span></li><li><span><a href="#Разбейте-данные-на-выборки" data-toc-modified-id="Разбейте-данные-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбейте данные на выборки</a></span></li><li><span><a href="#Исследуйте-модели" data-toc-modified-id="Исследуйте-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследуйте модели</a></span></li><li><span><a href="#Проверьте-модель-на-тестовой-выборке" data-toc-modified-id="Проверьте-модель-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверьте модель на тестовой выборке</a></span></li><li><span><a href="#(бонус)-Проверьте-модели-на-адекватность" data-toc-modified-id="(бонус)-Проверьте-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>(бонус) Проверьте модели на адекватность</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
print(df.shape)

(3214, 5)


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
df['calls'] = df['calls'].astype('int')
df['messages'] = df['messages'].astype('int')

**Вывод**

В данных нет пропущенных значений. Предобработка была проведена ранее. Признакам calls и messages тип данных изменен на int, т.к. это целочисленные данные.

## Разбейте данные на выборки

Создадим переменные features и target

In [7]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Создадим тестовые выборки

In [8]:
features, features_test, target, target_test = train_test_split(features, target, test_size= 0.2, random_state=12345)

Создадим обучающую и валидационную выборки

In [9]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.2, random_state=12345)

**Вывод**

Данные разбиты на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов  равны. Исходные данные разбиты в соотношении 3:1:1

## Исследуйте модели

**Дерево решений**

In [10]:
%%time
best_model_dtc = None
best_result_dtc = 0
best_depth_dtc = 0
for depth in range(1, 10):
    model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_split = 2) 
    model_dtc.fit(features_train, target_train)
    result_dtc = model_dtc.score(features_valid, target_valid)
    if result_dtc > best_result_dtc:
        best_model_dtc = model_dtc
        best_result_dtc = result_dtc
        best_depth_dtc = depth
print("Accuracy лучшей модели:", best_result_dtc)
print("Глубина дерева:", best_depth_dtc)


Accuracy лучшей модели: 0.7650485436893204
Глубина дерева: 7
CPU times: user 49.1 ms, sys: 2.9 ms, total: 52 ms
Wall time: 51.2 ms


**Случайный лес**

In [11]:
%%time
best_model_rfc = None
best_result_rfc = 0
best_est_rfc = 0
best_depth_rfc = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, min_samples_split = 2)
        model_rfc.fit(features_train, target_train)
        result_rfc = model_rfc.score(features_valid, target_valid)
        if result_rfc > best_result_rfc:
            best_model_rfc = model_rfc 
            best_result_rfc = result_rfc
            best_est_rfc = est
            best_depth_rfc = depth
print("Accuracy наилучшей модели на валидационной выборке:", best_result_rfc)
print("Количество деревьев:", best_est_rfc)
print("Глубина дерева:", best_depth_rfc)


Accuracy наилучшей модели на валидационной выборке: 0.7902912621359224
Количество деревьев: 50
Глубина дерева: 8
CPU times: user 3.47 s, sys: 30.2 ms, total: 3.5 s
Wall time: 3.52 s


**Логистическая регрессия**

In [12]:
%%time
model_lr = LogisticRegression(random_state=12345)
model_lr.fit(features_train, target_train) 
result_lr = model_lr.score(features_valid, target_valid) 
print("Accuracy логистической регрессии:", result_lr)

Accuracy логистической регрессии: 0.7165048543689321
CPU times: user 28.2 ms, sys: 4 µs, total: 28.2 ms
Wall time: 35.2 ms


**Вывод**

Быстрее всего сработала логистическая регрессия. Модель быстро обучается из-за  отсутствия гиперпараметров, но результат проверки модели на валидационной выборке худший - accuracy 0.72

На втором месте по скорости обучения - дерево решений. Модель обучалась дольше, но и результат лучше - accuracy 0.77.

Дольше всего обучалась модель случайный лес, зато она показала самый высокий результат - accuracy 0.79.

Суммарно, по показателям скорость обучения и accuracy, наиболее эффективна модель дерево решений - невысокая скорость обучения и довольно хороший результат.

## Проверьте модель на тестовой выборке

**Дерево решений**

In [13]:
%%time
test_result_dtc = best_model_dtc.score(features_test, target_test)
print('Accuracy модели дерево решений:', test_result_dtc)

Accuracy модели дерево решений: 0.7916018662519441
CPU times: user 3.91 ms, sys: 0 ns, total: 3.91 ms
Wall time: 2.83 ms


**Случайный лес**

In [14]:
%%time
test_result_rfc = best_model_rfc.score(features_test, target_test)
print('Accuracy модели случайный лес:', test_result_rfc)


Accuracy модели случайный лес: 0.8040435458786936
CPU times: user 8.66 ms, sys: 53 µs, total: 8.72 ms
Wall time: 8.27 ms


**Логистическая регрессия**

In [15]:
%%time
test_result_lr = model_lr.score(features_test, target_test)
print('Accuracy логистической регресии:', test_result_lr)

Accuracy логистической регресии: 0.76049766718507
CPU times: user 3.14 ms, sys: 0 ns, total: 3.14 ms
Wall time: 2.26 ms


**Вывод**

На тестовой выборке результаты следующие - у дерева решений самое короткое время и достаточно хороший accuracy, у случайного леса самый высокий accuracy и самое продолжительное время, у логистической регрессии - самый низкий accuracy и довольно короткое время обучения.

## (бонус) Проверьте модели на адекватность

Мы можем воспользоваться DummyClassifier, который всегда предсказывает мажоритарный класс. Дамми-классификатор, который генерирует случайные прогнозы, имеет намного худшее качество (с точки зрения правильности), чем рассмотренные выше модели.

In [16]:
%%time
model_dc = DummyClassifier(random_state=12345)
model_dc.fit(features_train, target_train)
result_dc = model_dc.score(features_valid, target_valid)
print('Accuracy  модели DummyClassifier:', result_dc)

Accuracy  модели DummyClassifier: 0.6757281553398058
CPU times: user 1.14 ms, sys: 98 µs, total: 1.23 ms
Wall time: 1.01 ms


**Вывод**

Результат случайного прогноза DummyClassifier ниже, чем у всех трех предыдущих моделей. Модели вполне адекватны.

## Вывод

Ниаболее эффективна модель - дерево решений - малое время обучения, достаточно высокий accuracy во всех расмотренных случаях. На вторм месте модель случайный лес - accuracy самый высокий, но и продолжительность самая высокая. На третьем - логистическая регрессия - достаточно малое время, но и результат низкий.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
